In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

X = tf.placeholder(tf.float32,[None,28,28,1]) # 28x28x1(Mono)
Y = tf.placeholder(tf.float32,[None,10]) # 0~9
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X,32,[3,3]) # 32개의 커널을 가진 윈도우 크기 3x3의 컨벌루션 계층 생성
L1 = tf.layers.max_pooling2d(L1,[2,2],[2,2]) # 커널 크기 2x2 의 풀링계층 만들고 슬라이딩시 2칸씩 이동(스트라이드).
L1 = tf.layers.dropout(L1,0.7,is_training) # Dropout 적용, 훈련 여부에 따라 해제

L2 = tf.layers.conv2d(L1,64,[3,3]) # 64개의 커널을 가진 윈도우 크기 3x3의 컨벌루션 계층 생성
L2 = tf.layers.max_pooling2d(L2,[2,2],[2,2]) # 커널 크기 2x2 의 풀링계층을 만들고 슬라이딩시 2칸씩 이동(스트라이드).
L2 = tf.layers.dropout(L2,0.7,is_training)

L3 = tf.contrib.layers.flatten(L2) # 완전 연결 계층 생성
L3 = tf.layers.dense(L3,256,activation=tf.nn.relu) # 256개의 뉴런으로 연결, 활성화 함수는 relu
L3 = tf.layers.dropout(L3,0.5,is_training)

model = tf.layers.dense(L3,10,activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        
        _, cost_val = sess.run([optimizer, cost],feed_dict={X:batch_xs,Y:batch_ys,is_training:True})
        total_cost += cost_val
        
    print('Epoch : {:04d}, Avg.cost : {:.3f}'.format(epoch+1, total_cost/total_batch))
    
print('최적화 완료 !')

is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('정확도 : {:.2f}%'.format(100*sess.run(accuracy,
        feed_dict={
        X:mnist.test.images.reshape(-1,28,28,1),
        Y:mnist.test.labels,
        is_training:False})))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch : 0001, Avg.cost : 0.174
Epoch : 0002, Avg.cost : 0.047
Epoch : 0003, Avg.cost : 0.029
Epoch : 0004, Avg.cost : 0.020
Epoch : 0005, Avg.cost : 0.015
Epoch : 0006, Avg.cost : 0.013
Epoch : 0007, Avg.cost : 0.011
Epoch : 0008, Avg.cost : 0.008
Epoch : 0009, Avg.cost : 0.011
Epoch : 0010, Avg.cost : 0.008
Epoch : 0011, Avg.cost : 0.006
Epoch : 0012, Avg.cost : 0.008
Epoch : 0013, Avg.cost : 0.007
Epoch : 0014, Avg.cost : 0.003
Epoch : 0015, Avg.cost : 0.006
최적화 완료 !
정확도 : 98.99%
